# HS04 model

## Phase 1

> The semantic component consisted of the 1,989 semantic features described above. These units were all connected to 50 units in the semantic cleanup apparatus...

- *50 sem_cleanup*

> The phonological representation consisted of the 200 phonolog-ical units (eight slots of 25 units each), which projected onto a set of 50 phonological cleanup units. These...

- *50 pho_cleanup*

> The semantic component mapped onto the phonological component via a set of 500 hidden units. There was feedback in both directions. 

- *500 sem_pho_hidden_units*
- *500 pho_sem_hidden_units*

> The phonological form of the target word was clamped on the phonological units for 2.66 units of time. Then a target signal was provided for the next 1.33 units of time, in which the network was required to retain the phonological pattern in the absence of external clamping. 

- *4 output_ticks* 

> In Harm and Seidenberg (1999), auto-connections were used to give the units a tendency to retain their value but gradually decay. To accomplish the task, the network had to learn enough of the statistical regularities of the representations to prevent this decay. In the current simulations, the idea is the same, but because continuous time units were used, auto-connections were not necessary to provide the units with a tendency to gradually decay; this was part of the units’ normal processing dynamics.

> HS99: This makes it easier to read weights as correlations between units. Each phonological unit has an auto-connection: a weight set to 0.75 and frozen to that value.

- *No auto-connection lock*

## Phase 2

> The weights that were obtained at the end of the Phase 1 model were frozen and embedded in the larger reading model. Thus, only the connections from orthography to other units were trained in Phase 2. Freezing the weights is not strictly necessary; earlier work (Harm & Seidenberg, 1997) used a process of intermixing in which comprehension trials were used along with reading trials. Weight freezing has the same effect but is simpler and less computationally burdensome to implement. Intermixing is effective and real- istic but adds substantially to network training time.

- *Pretraining is necessary (skip for now)*

> One set of 500 hidden units mediated the mapping from these orthographic units to semantics...

- *500 sem_hidden_units*

> ...a second set of 100 hidden units mediated the orth-phon pathway.

- *100 pho_hidden_units*

> To computationally instantiate the principle that the reading system is under pressure to perform rapidly as well as accurately, we injected error into the semantic and phonological representa- tions early, from time samples 2 to 12. 
- *11 output_ticks*

## Phase 3

- Modeling individual differences
- Simulating ERPs
- Link to reliance of OP vs OS
- Use equation to model semantic / phonetic input to P/S

In [ ]:
%reload_ext lab_black
import pickle, os
import tensorflow as tf
import numpy as np
import pandas as pd
from IPython.display import clear_output
from importlib import reload
import meta, data_wrangling, modeling




# Parameters

In [ ]:
code_name = "hs04_ps"
tf_root = "/home/jupyter/tf"

# Model architechture
ort_units = 119  # Phase 2 param (P2)
pho_units = 250
sem_units = 2446

hidden_os_units = 500  # P2
hidden_op_units = 100  # P2
hidden_ps_units = 500
hidden_sp_units = 500

pho_cleanup_units = 50
sem_cleanup_units = 50

pho_noise_level = 0.0  # P3
sem_noise_level = 0.0  # P3

activation = "sigmoid"
tau = 1 / 3
max_unit_time = 4.0
output_ticks = 4

# Training
sample_name = "hs04"
rng_seed = 53797
learning_rate = 0.005
n_mil_sample = 0.1
batch_size = 100
save_freq = 10

In [ ]:
config_dict = {}

# Load global cfg variables into a dictionary for feeding into ModelConfig()
for v in meta.CORE_CONFIGS:
    try:
        config_dict[v] = globals()[v]
    except:
        raise

for v in meta.OPTIONAL_CONFIGS:
    try:
        config_dict[v] = globals()[v]
    except:
        pass

# Construct ModelConfig object
cfg = meta.ModelConfig(**config_dict)
cfg.save()
del config_dict

# Build and compile model

In [ ]:
tf.random.set_seed(cfg.rng_seed)

data = data_wrangling.MyData()
model = modeling.HS04Phase1PS(cfg)

model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=cfg.learning_rate, beta_1=0.0, beta_2=0.999, amsgrad=False
    ),
    metrics=["BinaryAccuracy", "mse"],
)

# Train model

In [ ]:
my_sampling = data_wrangling.FastSampling(cfg, data)

history = model.fit(
    my_sampling.sample_generator(x="pho", y="sem"),
    steps_per_epoch=cfg.steps_per_epoch,
    epochs=cfg.total_number_of_epoch,
    verbose=1,
    # callbacks=[checkpoint],
)

with open(cfg.path["history_pickle"], "wb") as f:
    pickle.dump(history.history, f)

model.save(cfg.path["save_model_folder"])

# Evaluate model

In [ ]:
import altair as alt

evaluate.training_history(cfg.path["history_pickle"]).plot_all(
    save_file=os.path.join(cfg.path["plot_folder"], "training_history.html")
)

In [ ]:
# Strain
strain = evaluate.strain_eval(cfg, data, model)
strain.start_evaluate(
    output=os.path.join(cfg.path["model_folder"], "result_strain_item.csv")
)

# Grain
grain = evaluate.grain_eval(cfg, data, model)
grain.start_evaluate(
    output=os.path.join(cfg.path["model_folder"], "result_grain_item.csv")
)

# Taraban
taraban = evaluate.taraban_eval(cfg, data, model)
taraban.start_evaluate(
    output=os.path.join(cfg.path["model_folder"], "result_taraban_item.csv")
)

# Glushko
glushko = evaluate.glushko_eval(cfg, data, model)
glushko.start_evaluate(
    output=os.path.join(cfg.path["model_folder"], "result_glushko_item.csv")
)

# Create vis class for visualisation
vis = evaluate.vis(cfg.path["model_folder"])

## Strain and Grain

In [ ]:
sg = vis.plot_dev_interactive("acc", ["strain", "grain"]).properties(
    title="Accuracy in Strain and Grain"
)
sg.save(os.path.join(cfg.path["plot_folder"], "development_sg.html"))
sg.display()

### Grain response

In [ ]:
small = vis.plot_dev_interactive("acc_small_grain", exp=["grain"]).properties(
    title="Small Grain Response"
)
large = vis.plot_dev_interactive("acc_large_grain", exp=["grain"]).properties(
    title="Large Grain Response"
)
grain_plot = (small | large).properties(
    title="Accuracy of Grain by response and condition"
)
grain_plot.save(
    os.path.join(cfg.path["plot_folder"], "development_grain_by_response.html")
)
grain_plot.display()